In [2]:
!pip install pyspark==3.5.3

In [3]:
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_1.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_2.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_3.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_4.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_5.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_6.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_7.log.gz

--2024-11-14 21:37:30--  https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_1.log.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10277393 (9.8M) [application/octet-stream]
Saving to: ‘sample_web_log_1.log.gz’

sample_web_log_1.lo 100%[===================>]   9.80M  --.-KB/s    in 0.1s    

2024-11-14 21:37:30 (68.8 MB/s) - ‘sample_web_log_1.log.gz’ saved [10277393/10277393]

--2024-11-14 21:37:30--  https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_2.log.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

Configure snowflake jar file and set up SparkSession

In [4]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar

--2024-11-14 21:37:38--  https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70986770 (68M) [application/java-archive]
Saving to: ‘snowflake-jdbc-3.19.0.jar’

snowflake-jdbc-3.19 100%[===================>]  67.70M   171MB/s    in 0.4s    

2024-11-14 21:37:39 (171 MB/s) - ‘snowflake-jdbc-3.19.0.jar’ saved [70986770/70986770]



In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("HandleLogFiles").getOrCreate()

In [6]:
# Load all .gz files in the directory into a DataFrame
df = spark.read.text("*.gz")

In [ ]:
# Check the number of partitions
print(df.rdd.getNumPartitions())

df.show(truncate=False)

3
+-----------------------------------------------------------------------------------+
|value                                                                              |
+-----------------------------------------------------------------------------------+
|123.45.67.89 - - [05/Nov/2024:02:08:16 +0000] "DELETE /cart HTTP/1.1" 500 242      |
|192.168.1.1 - - [04/Nov/2024:21:23:39 +0000] "POST /checkout HTTP/1.1" 404 2781    |
|234.56.78.90 - - [05/Nov/2024:07:06:19 +0000] "GET /api/data HTTP/1.1" 301 3758    |
|192.168.1.1 - - [04/Nov/2024:20:03:56 +0000] "POST /home HTTP/1.1" 200 1837        |
|192.168.1.1 - - [04/Nov/2024:21:25:05 +0000] "GET /products/123 HTTP/1.1" 200 3430 |
|234.56.78.90 - - [04/Nov/2024:07:38:10 +0000] "GET /api/data HTTP/1.1" 404 3729    |
|123.45.67.89 - - [04/Nov/2024:12:33:22 +0000] "PUT /api/data HTTP/1.1" 404 799     |
|192.168.1.1 - - [04/Nov/2024:07:37:46 +0000] "GET /api/data HTTP/1.1" 500 309      |
|123.45.67.89 - - [04/Nov/2024:21:52:36 +0000] "POST

In [7]:
# Extract the necessary information from log data using regular expressions
pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?) (.*?) HTTP.*" (\d+) (\d+)'

log_df = df.select(
    F.regexp_extract("value", pattern, 1).alias("ip"),
    F.regexp_extract("value", pattern, 2).alias("timestamp"),
    F.regexp_extract("value", pattern, 3).alias("method"),
    F.regexp_extract("value", pattern, 4).alias("url"),
    F.regexp_extract("value", pattern, 5).alias("status").cast("integer"),
    F.regexp_extract("value", pattern, 6).alias("size").cast("integer")
)

In [8]:
log_df.show()

+------------+--------------------+------+-------------+------+----+
|          ip|           timestamp|method|          url|status|size|
+------------+--------------------+------+-------------+------+----+
|123.45.67.89|05/Nov/2024:02:08...|DELETE|        /cart|   500| 242|
| 192.168.1.1|04/Nov/2024:21:23...|  POST|    /checkout|   404|2781|
|234.56.78.90|05/Nov/2024:07:06...|   GET|    /api/data|   301|3758|
| 192.168.1.1|04/Nov/2024:20:03...|  POST|        /home|   200|1837|
| 192.168.1.1|04/Nov/2024:21:25...|   GET|/products/123|   200|3430|
|234.56.78.90|04/Nov/2024:07:38...|   GET|    /api/data|   404|3729|
|123.45.67.89|04/Nov/2024:12:33...|   PUT|    /api/data|   404| 799|
| 192.168.1.1|04/Nov/2024:07:37...|   GET|    /api/data|   500| 309|
|123.45.67.89|04/Nov/2024:21:52...|  POST|    /checkout|   301|2375|
|123.45.67.89|04/Nov/2024:08:36...|DELETE|    /api/data|   404|3449|
| 192.168.1.1|05/Nov/2024:03:15...|   GET|    /api/data|   200|2319|
|234.56.78.90|05/Nov/2024:01:26...

In [9]:
#compute count of each unique URL and Status combination
count_df=log_df.groupby("url","status").count()
count_df.show()

+-------------+------+------+
|          url|status| count|
+-------------+------+------+
|/products/123|   500|349853|
|/products/123|   200|349536|
|    /checkout|   500|349414|
|/products/123|   301|349904|
|    /checkout|   200|351036|
|        /home|   500|351457|
|        /cart|   500|349433|
|        /home|   404|349492|
|    /checkout|   404|349604|
|        /cart|   301|350262|
|    /api/data|   200|349703|
|    /api/data|   301|348954|
|    /api/data|   500|350091|
|        /cart|   200|350305|
|        /home|   200|350541|
|/products/123|   404|350970|
|        /home|   301|350203|
|    /checkout|   301|349914|
|        /cart|   404|349830|
|    /api/data|   404|349498|
+-------------+------+------+



In [16]:
# Register the DataFrame as a temporary SQL table
log_df.createOrReplaceTempView("logs_df_status")
# Use SparkSQL to count URLs with 404 status
count_df_spark = spark.sql("""
    SELECT url, status,COUNT(*) as count_status
    FROM logs_df_status
    GROUP BY url, status
    ORDER BY count_status DESC
""")
count_df_spark.show()

+-------------+------+------------+
|          url|status|count_status|
+-------------+------+------------+
|        /home|   500|      351457|
|    /checkout|   200|      351036|
|/products/123|   404|      350970|
|        /home|   200|      350541|
|        /cart|   200|      350305|
|        /cart|   301|      350262|
|        /home|   301|      350203|
|    /api/data|   500|      350091|
|    /checkout|   301|      349914|
|/products/123|   301|      349904|
|/products/123|   500|      349853|
|        /cart|   404|      349830|
|    /api/data|   200|      349703|
|    /checkout|   404|      349604|
|/products/123|   200|      349536|
|    /api/data|   404|      349498|
|        /home|   404|      349492|
|        /cart|   500|      349433|
|    /checkout|   500|      349414|
|    /api/data|   301|      348954|
+-------------+------+------------+



In [13]:
# Keep only 404 error logs
error_404_logs = log_df.filter(log_df.status == 404)

In [14]:
# Group by URL and then count, and sort by count in descending order
url_404_count = error_404_logs.groupBy("url").count().orderBy(F.desc("count"))

In [15]:
# print the outcome
url_404_count.show()

+-------------+------+
|          url| count|
+-------------+------+
|/products/123|350970|
|        /cart|349830|
|    /checkout|349604|
|    /api/data|349498|
|        /home|349492|
+-------------+------+



In [ ]:
# Register the DataFrame as a temporary SQL table
log_df.createOrReplaceTempView("logs")

In [17]:
# Use SparkSQL to count URLs with 404 status
url_404_count = spark.sql("""
    SELECT url, COUNT(1) as count
    FROM logs
    WHERE status = 404
    GROUP BY url
    ORDER BY count DESC
""")

In [ ]:
url_404_count.show()

+-------------+------+
|          url| count|
+-------------+------+
|/products/123|350970|
|        /cart|349830|
|    /checkout|349604|
|    /api/data|349498|
|        /home|349492|
+-------------+------+



In [28]:
from google.colab import userdata

account = userdata.get('account')
user = userdata.get('User')
password = userdata.get('snowflake_password')
database = "dev"
schema = "analytics"

url = f"jdbc:snowflake://{account}.snowflakecomputing.com/?db={database}&schema={schema}&user={user}&password={password}"

In [30]:
count_df_spark.write \
    .format("jdbc") \
    .option("driver", "net.snowflake.client.jdbc.SnowflakeDriver") \
    .option("url", f"jdbc:snowflake://{account}.snowflakecomputing.com") \
    .option("dbtable", "count_df_spark") \
    .option("user", user) \
    .option("password", password) \
    .option("database", database) \
    .option("schema", schema) \
    .mode("overwrite") \
    .save()